In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

In [2]:
df = pd.read_csv('train.csv')
n_users = df.userID.unique().shape[0]
n_items = df.loc[:,['itemID']].max()['itemID']
print('number of user is '+ str(n_users) + ' | Number of item = ' + str(n_items))

number of user is 2967 | Number of item = 4124


In [3]:
from sklearn import model_selection as ms
train_data,test_data = ms.train_test_split(df, test_size = 0.25)
train_data_matrix = np.zeros((n_users,n_items))
test_data_matrix = np.zeros((n_users,n_items))
#print(train_data_matrix.shape)
for line in train_data.itertuples():
    #print(line)
    train_data_matrix[int(line[1])-1, int(line[2])-1] = line[3]
for line in test_data.itertuples():
    #print(line)
    test_data_matrix[int(line[1])-1, int(line[2])-1] = line[3]
train_data_matrix[:,0] += 1e-9;
train_data_matrix[0] += 1e-9
test_data_matrix[:,0] += 1e-9;
test_data_matrix[0] += 1e-9;

In [4]:
import time
from tqdm import tqdm

In [5]:
connect_user_item = np.zeros((n_users,n_items))
#connect_user_item[connect_user_item != 0] = 1;
for i in range(0, n_users):
    for j in range(0, n_items):
        if(train_data_matrix[i,j] != 0):
            connect_user_item[i,j] = 1;
print(connect_user_item.shape)
#print(train_data_matrix)

(2967, 4124)


In [6]:
n_features = 3;#参数
f_users = np.full((n_users, n_features),0.5)
f_items = np.full((n_items, n_features),0.5)
landar = 0.5
#print(type(f_users[0]))
#print(f_users.shape)
#print(sum(f_users[2]))

In [7]:
#tmp1 = np.zeros((n_users, n_features))
#tmp2 = np.zeros(n_users)
#for i in tqdm(range(0, n_users)):
#    for j in range(0, n_items):
#       if(train_data_matrix[i,j] != 0):
#            tmp1[i] = tmp1[i] + train_data_matrix[i][j] * f_items[i]
#            tmp2[i] = tmp2[i] + np.dot(f_items[j], f_items[j])
#for i in tqdm(range(0, n_users)):
#    f_users[i] = tmp1[i] / ( tmp2[i]+landar)

In [8]:
def count_f_users():
    #f_users = np.full((n_users, n_features),0.5)
    #f_items = np.full((n_items, n_features),0.5)
    tmp1 = np.zeros((n_users, n_features))
    tmp2 = np.zeros(n_users)
    for i in range(0, n_users):
        for j in range(0, n_items):
            if(train_data_matrix[i,j] != 0):
                tmp1[i] = tmp1[i] + train_data_matrix[i][j] * f_items[i]
                tmp2[i] = tmp2[i] + np.dot(f_items[j], f_items[j])
    for i in tqdm(range(0, n_users)):
        f_users[i] = tmp1[i] / ( tmp2[i]+landar)
    return f_users

In [9]:
#print(count_f_users(n_users, n_items, 3, ))

In [10]:
#tmp3 = np.zeros((n_items, n_features))
#tmp4 = np.zeros(n_items)
#for i in tqdm(range(0, n_users)):
#    for j in range(0, n_items):
#        if(train_data_matrix[i,j] != 0):
#            tmp3[j] = tmp3[j] + train_data_matrix[i,j] * f_users[i]
#            tmp4[j] = tmp4[j] + np.dot(f_users[i], f_users[i])
#for i in tqdm(range(0, n_items)):
#    f_items[i] = tmp3[i] / ( tmp4[i] + landar)

In [11]:
def count_f_items():
    #f_items = np.full((n_items, n_features),0.5)
    #f_users = np.full((n_users, n_features),0.5)
    tmp3 = np.zeros((n_items, n_features))
    tmp4 = np.zeros(n_items)
    for i in range(0, n_users):
        for j in range(0, n_items):
            if(train_data_matrix[i,j] != 0):
                tmp3[j] = tmp3[j] + train_data_matrix[i,j] * f_users[i]
                tmp4[j] = tmp4[j] + np.dot(f_users[i], f_users[i])
    for i in range(0, n_items):
        f_items[i] = tmp3[i] / ( tmp4[i] + landar)
    return f_items

In [12]:
#print(count_f_items(n_users, n_items, 3))

In [13]:
k = 20
while(k > 0):
    k -= 1
    f_users = count_f_users()
    f_items = count_f_items()

100%|██████████| 2967/2967 [00:00<00:00, 135216.33it/s]


In [14]:
print(f_users)
print(f_items)

[[4.70023316e-06 4.70023316e-06 4.70023316e-06]
 [8.31060896e-02 8.31060896e-02 8.31060896e-02]
 [1.64939007e-01 1.64939007e-01 1.64939007e-01]
 ...
 [1.06541015e+01 1.06541015e+01 1.06541015e+01]
 [1.81643429e+02 1.81643429e+02 1.81643429e+02]
 [1.14589178e+01 1.14589178e+01 1.14589178e+01]]
[[3.67601793e-04 3.67601793e-04 3.67601793e-04]
 [1.30538852e-03 1.30538852e-03 1.30538852e-03]
 [1.42166951e-03 1.42166951e-03 1.42166951e-03]
 ...
 [3.55408501e-13 3.55408501e-13 3.55408501e-13]
 [9.40046631e-15 9.40046631e-15 9.40046631e-15]
 [1.96360958e-03 1.96360958e-03 1.96360958e-03]]


In [15]:
prediction = np.zeros(n_users,n_items)
prediction = np.dot(f_users,f_items.T)
prediction[prediction < 0.5] = 0.5
prediction[prediction > 5] = 5
print(prediction)

[[0.5        0.5        0.5        ... 0.5        0.5        0.5       ]
 [0.5        0.5        0.5        ... 0.5        0.5        0.5       ]
 [0.5        0.5        0.5        ... 0.5        0.5        0.5       ]
 ...
 [0.5        0.5        0.5        ... 0.5        0.5        0.5       ]
 [0.5        0.71134574 0.77471077 ... 0.5        0.5        1.07003033]
 [0.5        0.5        0.5        ... 0.5        0.5        0.5       ]]


In [16]:
test_df = pd.read_csv("test_index.csv")
test_data_matrix = test_df.values

predict_matrix = np.zeros((39679, 2))
f = open('pmf_predict.csv','w')
f.write('dataID,rating\n')
i = 0
for i in range(test_data_matrix.shape[0]):
    predict_matrix[i][0] = int(i)
    predict_matrix[i][1] = prediction[test_data_matrix[i][0]][test_data_matrix[i][1]]
    f.write('%d,%.8f\n'%(predict_matrix[i][0],predict_matrix[i][1]))
#print(predict_matrix)


# predict_df = pd.DataFrame({"dataID": predict_matrix.T[0], "rating": predict_matrix.T[1]})
# predict_df.to_csv("predict.csv", index=False)

In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [18]:
#delta = 1
# best_k = -1
# result_with_test = 100
# best_result = 100
# for i in tqdm(range(10,20)):
#     while(delta > 0.1):
#         print(i)
#         f_users = np.zeros((n_users, i))
#         f_items = np.zeros((n_items, i))
#         f_users = count_f_users(n_users, n_items, i)
#         f_items = count_f_items(n_users, n_items, i)
#         prediction = np.dot(f_users, f_items.T)
#         delta = rmse(prediction, train_data_matrix)
#         print(delta)
#     result_with_test = rmse(prediction, test_data_matrix)
#     if(result_with_test < best_result):
#         best_result = result_with_test
#         best_k = i
#         print("best_k changed " + i)
# print(i)

In [19]:
#print(i)